In [2]:
%pylab inline
from SimPEG import DCIP as DC

Populating the interactive namespace from numpy and matplotlib


ImportError: cannot import name DCIP

In [ ]:
"../data/ChungCheonDC/"

In [ ]:

def readReservoirDC(fname):
    f = open(fname, 'r')
    data = f.readlines()
    temp = data[3].split()
    nelec, ndata, aspacing = int(temp[0]), int(temp[1]), float(temp[2])
    ntx = nelec-2
    datalist = []
    for iline, line in enumerate(data[4:4+ndata]):
    #     line = line.replace(ignorevalue, 'nan')
        linelist = line.split()    
        datalist.append(np.array(map(float, linelist)))
    DAT = np.vstack(datalist)
    datalistSRC = []
    srcList = []
#     for i in range(ntx-1):
    for i in range(ntx-1):        
        txloc = np.array([i+2, i+1.])
        ind = (DAT[:,:2] == txloc).sum(axis=1) == 2.
        temp = DAT[ind,:]
        datalistSRC.append(temp)
        e = np.zeros_like(temp[:,2])
        rxtemp = DC.RxDipole(np.c_[temp[:,2]*aspacing, e, e], np.c_[temp[:,3]*aspacing, e, e])
        srctemp = DC.SrcDipole([rxtemp], np.r_[txloc[1]*aspacing, 0., 0.], np.r_[txloc[0]*aspacing, 0., 0.])
        srcList.append(srctemp)
    DAT_src = np.vstack(datalistSRC)    
    survey = DC.SurveyDC(srcList)
    survey.dobs = DAT_src[:,-1]
    return survey

In [ ]:
fname1 = "../data/ChungCheonDC/20150328120000.apr"
fname2 = "../data/ChungCheonDC/20150328180000.apr"

In [ ]:
survey1 = readReservoirDC(fname1)
survey2 = readReservoirDC(fname2)
survey12 = readReservoirDC(fname2)

In [ ]:
# ind = (survey1.dobs > 200.) | (survey2.dobs > 200.)
# survey1.dobs[ind] = np.median(survey1.dobs)
# survey2.dobs[ind] = np.median(survey2.dobs)

In [ ]:
survey12.dobs = abs(survey2.dobs-survey1.dobs) / abs(survey1.dobs)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (12*2, 3*2))
DC.plot_pseudoSection(survey1, ax, "dpdp", dtype="appr", clim=[52, 150.])

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (12*2, 3*2))
DC.plot_pseudoSection(survey2, ax, "dpdp", dtype="appr", clim=[52, 150.])

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (12*2, 3*2))
DC.plot_pseudoSection(survey12, ax, "dpdp", dtype="appr", clim=[0., 0.1])